In [1]:
import os
import warnings
from dotenv import load_dotenv

#If you get duplicate error run---

#os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
#warnings.filterwarnings("ignore")
# ----


env_path = "../OllamaSetup/.env"  
load_dotenv(env_path)

True

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

In [3]:
llm= ChatOllama(model='llama3.2:3b',base_url='http://localhost:11434')
llm.invoke("Hi")

AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2025-04-02T03:25:29.656598Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3653426792, 'load_duration': 742025042, 'prompt_eval_count': 26, 'prompt_eval_duration': 2722011750, 'eval_count': 8, 'eval_duration': 188194208, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-06a16057-076a-4263-87dd-88d71379310e-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34})

## Define Tools

## Tavily

In [4]:
from langchain_community.tools import TavilySearchResults


In [5]:
@tool
def search(query:str) -> str :
    """ 
    Search the web for real time and latest information.
    for example news,weather upadate ,stock news etc

    Args: The search query   
    """

    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )

    response = search.invoke(query)
    return response

## Health Supplement Retriever

In [6]:
from langchain_ollama import OllamaEmbeddings
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [7]:
embeddings = OllamaEmbeddings(model= 'nomic-embed-text:latest',base_url='http://localhost:11434')
db_name ="/Users/ragnirupam/Desktop/GenAIProjects/genaienv/VectorStoreAndRetrievals/health_supplements"
#db_name= "health_supplements"
vector_store= FAISS.load_local(db_name,embeddings, allow_dangerous_deserialization=True)

In [8]:
retriever =vector_store.as_retriever(search_type ='similarity',search_kwargs={'k':3})

retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10bb67610>, search_kwargs={'k': 3})

In [9]:
@tool
def health_supplement(query:str) -> str :
    """ Search for the information about Health Supplements.
    For any question about Health and gym supplements , you must use this tool !,
     
    Args:
        query: The search query.
     """
    
    response= retriever.invoke(query)
    return response

In [10]:
health_supplement.invoke("How Can you loose weight")

[Document(id='85c771b9-02c8-4d6d-bc7f-5e79a2386f40', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'file_path': 'rag-dataset/health supplements/1. dietary supplements - for whom.pdf', 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:37:54+00:00', 'trapped': '', 'modDate': 'D:20241021113754Z', 'creationDate': '', 'page': 12}, page_content='Int. J. Environ. Res. Public Health 2021, 18, 8897\n13 of 24\n3.3. Dietary Supplements and Weight Loss\nThe number of people with excessive weight continues to rise, and ﬁghting obesity\nhas become one of the greatest challenges of contemporary medicine. A person wishing to\nlose weight needs to undertake several difﬁcult life-changes and practice them consistently\n(diet, physical activity, addiction-free). Meanwhile dietary supplements are presented as\na compelling altern

## Agent Creation

In [11]:
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [12]:
prompt= hub.pull("hwchase17/openai-functions-agent")

In [13]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [14]:
tools= [search,health_supplement]
agent = create_tool_calling_agent(llm,tools,prompt)

In [15]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMes

In [16]:
agent_executor= AgentExecutor(agent = agent,tools= tools, verbose=True)

In [17]:
question="What is the best supplement for muscle gain ? "
response = agent_executor.invoke({'input' : question})




> Entering new AgentExecutor chain...

Invoking: `health_supplement` with `{'query': 'best supplement for muscle gain'}`


[Document(id='2b4342ba-48bd-4b0e-9cd5-d8b155987ed7', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'rag-dataset/gym supplements/1. Analysis of Actual Fitness Supplement.pdf', 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:38:50+00:00', 'trapped': '', 'modDate': 'D:20241021113850Z', 'creationDate': '', 'page': 0}, page_content='acids than traditional protein sources. Its numerous benefits have made it a popular choice\nfor snacks and drinks among consumers [3]. Another widely embraced supplement is\ncaffeine, which is found in many sports and food supplements. Caffeine reduces perceived\neffort, minimizes fatigue and pain, and proves to be effective for endurance and h

In [18]:
response

{'input': 'What is the best supplement for muscle gain ? ',
 'output': "Based on the analysis of actual fitness supplements, it appears that several supplements can be effective for muscle gain. Some of the most popular and well-researched options include:\n\n1. Creatine monohydrate: This supplement has been consistently shown to increase muscle mass and support performance and recovery.\n2. Branched-chain amino acids (BCAA): BCAAs are stored directly in muscles and serve as the raw materials needed to build new muscle, contributing to strengthening muscles and alleviating post-workout soreness.\n3. Protein powder: High-quality protein powder can help meet nutritional needs for muscle growth and repair.\n\nIt's essential to note that individual results may vary, and it's always recommended to consult with a healthcare professional or registered dietitian before adding any new supplements to your regimen. Additionally, a balanced diet and regular exercise are crucial for achieving optim

In [19]:
print(response['output'])

Based on the analysis of actual fitness supplements, it appears that several supplements can be effective for muscle gain. Some of the most popular and well-researched options include:

1. Creatine monohydrate: This supplement has been consistently shown to increase muscle mass and support performance and recovery.
2. Branched-chain amino acids (BCAA): BCAAs are stored directly in muscles and serve as the raw materials needed to build new muscle, contributing to strengthening muscles and alleviating post-workout soreness.
3. Protein powder: High-quality protein powder can help meet nutritional needs for muscle growth and repair.

It's essential to note that individual results may vary, and it's always recommended to consult with a healthcare professional or registered dietitian before adding any new supplements to your regimen. Additionally, a balanced diet and regular exercise are crucial for achieving optimal muscle gain.

Sources:

[1] Foods 2024, 13, 1424
[2] https://doi.org/10.339

In [20]:
question="How is the weather in NewYork ? "
response = agent_executor.invoke({'input' : question})




> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'New York weather'}`


SSLError(MaxRetryError('HTTPSConnectionPool(host=\'api.tavily.com\', port=443): Max retries exceeded with url: /search (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for \'api.tavily.com\'. (_ssl.c:1002)")))'))Unfortunately, I was unable to retrieve the current weather in New York due to a SSL error. However, I can suggest some alternative ways to find the current weather in New York:

1. Check online weather websites such as accuweather.com, weather.com, or wunderground.com for the most up-to-date weather information.
2. Use a virtual assistant like Siri, Google Assistant, or Alexa to ask for the current weather in New York.
3. Check your local news website or app for the latest weather forecast.

I apologize for the inconvenience and hope this helps!

> Finished chain.


In [21]:
print(response['output'])

Unfortunately, I was unable to retrieve the current weather in New York due to a SSL error. However, I can suggest some alternative ways to find the current weather in New York:

1. Check online weather websites such as accuweather.com, weather.com, or wunderground.com for the most up-to-date weather information.
2. Use a virtual assistant like Siri, Google Assistant, or Alexa to ask for the current weather in New York.
3. Check your local news website or app for the latest weather forecast.

I apologize for the inconvenience and hope this helps!
